In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
iris=pd.read_fwf('train.ft.txt',header=None)

In [3]:
data=iris.copy()
data=data.dropna(axis=1)

In [4]:
def f(s):
    if s=="__label__2":
        return 'good'
    else:
        return 'bad'

In [5]:
data.columns=["verdict","text"]
    

In [6]:
data.verdict=data.verdict.apply(f)

In [7]:
data=data.sample(frac=1).reset_index(drop=True)

In [8]:
iris=data.copy()

In [9]:
data=iris.iloc[0:40000].copy()

In [10]:
def removeemoji(text):
    p=''.join(t if ord(t)<128 else '' for t in text)
    return p

In [11]:
data['text']=data['text'].apply(removeemoji)

In [12]:
from nltk.stem import WordNetLemmatizer
lem=WordNetLemmatizer()

In [13]:
from nltk.corpus import stopwords
stops=stopwords.words('english')
import string
stops+=[w for w in string.punctuation]

In [14]:
from nltk import word_tokenize
from nltk import pos_tag


In [15]:
from nltk.corpus import wordnet
def get_simple_pos(word):
    if word.startswith('J'):
        return wordnet.ADJ
    if word.startswith('V'):
        return wordnet.VERB
    if word.startswith('N'):
        return wordnet.NOUN
    if word.startswith('R'):
        return wordnet.ADV
    return wordnet.NOUN

In [16]:
qwe=data.copy()

In [17]:
count=1
total=200000
import IPython
from IPython.display import clear_output

In [18]:
def removestops(text):
    global count
    global total
    count=count+1
    if count%200==0:
        clear_output()
        print((count*100)/total)
    
    letters_only = re.sub("[^a-zA-Z]", " ",text) 
    words = letters_only.lower().split()                  
    meaningful_words = ''
    for w in words:
        if w not in stops:
            p=w
            p=lem.lemmatize(w,pos=get_simple_pos(pos_tag([w])[0][1]))
            meaningful_words+=p.lower()+' '
    return meaningful_words

In [19]:
data.text=data.text.apply(removestops)

20.0


In [20]:
x=data.text
y=data.verdict
xx=[]
for i in range(len(x)):
    xx.append(x[i])


In [21]:
from sklearn.model_selection import train_test_split
xt,xtest,yt,ytest=train_test_split(xx,y,test_size=0.33,random_state=1)

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
vec=TfidfVectorizer(ngram_range=(1,4))
tx=vec.fit_transform(xt)

In [23]:
test_y=vec.transform(xtest)

In [24]:
from sklearn.naive_bayes import MultinomialNB
clf=MultinomialNB()
clf.fit(tx,yt)
print(clf.score(test_y,ytest))
ans=clf.predict(test_y)

0.855909090909091


In [25]:
from sklearn.metrics import confusion_matrix
print(confusion_matrix(ytest,ans))
from sklearn.metrics import classification_report
print(classification_report(ans , ytest))

[[5750  779]
 [1123 5548]]
              precision    recall  f1-score   support

         bad       0.88      0.84      0.86      6873
        good       0.83      0.88      0.85      6327

    accuracy                           0.86     13200
   macro avg       0.86      0.86      0.86     13200
weighted avg       0.86      0.86      0.86     13200

